In [50]:
from langchain_ollama import OllamaLLM
import langchain
import numpy
import pandas


#initialize model
lafilamod= OllamaLLM(
    model="Llama3:latest",
    temperature=0,
    num_ctx=2048,
    verbose=True,
    
)

In [6]:
from PyPDF2 import PdfReader

# Read pdf, get text
reader = PdfReader('/Users/abhimanyu/Downloads/Hands-On_Large_Language_Models_compressed.pdf')
page_ids = []
page_texts = []
for i, p in enumerate(reader.pages):
    page_ids.append(str(i))
    page_texts.append(p.extract_text())



In [7]:
print(page_ids)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '15

In [8]:
# from langchain.text_splitter import MarkdownTextSplitter

# # Create a MarkdownTextSplitter for semantic chunking
# semantic_splitter = MarkdownTextSplitter(
#     chunk_size=200,    # Number of characters per chunk
#     chunk_overlap=50   # Overlap to preserve context
# )

# # Split the text into semantic chunks
# semantic_chunks = semantic_splitter.split_text(page_texts)
# print("\nSemantic Chunks (Markdown-based):\n", semantic_chunks)


In [9]:
# #now embeddings for the text
# from sentence_transformers import SentenceTransformer

# embed_model=SentenceTransformer("")

# text_embeddings=embed_model.encode(page_texts)

# index_embeddings=embed_model.encode(page_ids)
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
embed_model=HuggingFaceBgeEmbeddings(model_name='thenlper/gte-small')

from langchain.vectorstores import FAISS
metadata= [{"page":int(i)} for i in page_ids]
database= FAISS.from_texts(page_texts, embed_model, metadatas=metadata)


/var/folders/ql/hdbyk3lx6vq0x6fs9227fc_m0000gn/T/ipykernel_50628/1651442095.py:10: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model=HuggingFaceBgeEmbeddings(model_name='thenlper/gte-small')
/Users/abhimanyu/Desktop/geospacy/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W0804 09:57:47.185000 50628 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
/Users/abhimanyu/Desktop/geospacy/

In [58]:
from langchain.prompts import PromptTemplate

#query rewriting
# def querewrite(query):
rewtemp=(
        "You are an AI assistant. Rewrite the following user query into a clear, concise search query suitable for retrieving relevant documents. "
        "User Query:+ {query} \nRewritten Query:"
    )
rewprompt= PromptTemplate(template= rewtemp, input_variables=['query'])



In [60]:
chain1=  rewprompt | lafilamod

In [61]:

document_prompt = PromptTemplate(
    input_variables=["page_content"],
     template="{page_content}"
)

In [62]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import RegexParser
parser= RegexParser(
    regex=r"answer: (.*?)\nScore:(.*)",
    output_keys=["answer","score"],
)
# template1="""
# <|begin_of_text|><|start_header_id|>system<|end_header_id|>
# you are a search retrieval agent that answers precisely to the query.the format needs to be like:
# Answer: <some answer>
# Score: <number between 0 and 1>
# <|eot_id|>
# <|start_header_id|>user<|end_header_id|>
# provide a relevant answer according to the user query:
# {context}
# the format needs to be like:
# Answer: <some answer>
# Score: <number between 0 and 1>

# <|eod_id|>
# <|start_header_id|>assistant<|end_header_id|>"""

from langchain.prompts import PromptTemplate

# rag_prompt = PromptTemplate.from_template("""
# Given the following document and a question, provide a helpful answer and a score between 0 and 1 indicating how well the document answers the question.

# Document:
# {context}

# Question:
# {question}

# Answer the question and rate the relevance.

# Output format:
# Answer: <your answer>
# Score: <score between 0 and 1>
# """)
template1map="""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an intelligent assistant helping to answer a user's question using a provided document.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
provide a relevant answer according to the user query:
{question}.

Document: {summaries}

the format needs to be like:
Answer: <some answer>
Score: <number between 0 and 1>

<|eod_id|>
<|start_header_id|>assistant<|end_header_id|>"""

template1combine="""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a smart assistant combining multiple partial answers from different documents to answer a user’s question.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
question:{question}
partial answers: {summaries}

based on the above construct a concrete final answer for the user:
<|eod_id|>
<|start_header_id|>assistant<|end_header_id|>"""

map_prompt= PromptTemplate(template=template1map, input_variables=[ "question","summaries"])
combine_prompt= PromptTemplate(template=template1combine, input_variables=["question", "summaries"])
document_variable_name="summaries"
# from langchain.chains import RetrievalQA
# rag= RetrievalQA.from_chain_type(llm=lafilamod, chain_type="map_rerank", retriever= database.as_retriever(), chain_type_kwargs={'prompt': rag_prompt},output_parser=parser, verbose=True,)

In [22]:
from langchain.chains import RetrievalQA, LLMChain, MapReduceDocumentsChain, StuffDocumentsChain, ReduceDocumentsChain

# rag= RetrievalQA.from_chain_type(llm=lafilamod, chain_type="map_reduce", retriever= database.as_retriever(),chain_type_kwargs={'map_prompt': map_prompt,"combine_prompt": combine_prompt})

llm_chain_map= LLMChain(llm=lafilamod, prompt= map_prompt)
llm_chain_combine=LLMChain(llm=lafilamod, prompt=combine_prompt)

combine_docschain=StuffDocumentsChain(llm_chain=llm_chain_combine,     document_prompt=document_prompt,
document_variable_name=document_variable_name)

reduce_doc_chain=ReduceDocumentsChain(combine_documents_chain=combine_docschain)

map_reduce_chain= MapReduceDocumentsChain(
    llm_chain= llm_chain_map,
    reduce_documents_chain= reduce_doc_chain,
    document_variable_name=document_variable_name
)
from langchain.chains import RetrievalQA

rag= RetrievalQA( retriever= database.as_retriever(), combine_documents_chain= map_reduce_chain )

In [63]:

chain2= rag | chain1

In [67]:
rag_response=chain2.invoke("what is an llm agent?")
print(rag_response)




> Entering new RetrievalQA chain...

> Finished chain.
Here's a rewritten version of your query that can help retrieve relevant documents:

**Search Query:** "Large Language Model (LLM) Agent Definition"

This rewritten query uses specific keywords and phrases to help search engines or databases quickly identify relevant documents. The use of parentheses around "Large Language Model" helps to clarify the meaning of LLM, which is a type of AI model.


In [47]:
template1="""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
you are a search retrieval agent that answers precisely to the query. 
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
provide a relevant answer according to the user query:
{question}
Relevant text:
{context}

<|eod_id|>
<|start_header_id|>assistant<|end_header_id|>"""

stuff_prompt=PromptTemplate(template= template1, input_variables=["question","context"])
ragstuff= RetrievalQA.from_chain_type(llm=lafilamod, chain_type="stuff", retriever= database.as_retriever(), chain_type_kwargs={'prompt': stuff_prompt, 'document_variable_name': 'context'}, verbose=True,)

In [44]:
def q(question):
    response= ragstuff.invoke(question)
    return response


In [51]:
q("what is an llm agent, how to use it for search retrieval")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'what is an llm agent, how to use it for search retrieval',
 'result': '**What is an LLM Agent?**\n\nAn LLM (Large Language Model) agent is a system that leverages a language model to determine its actions and make decisions. It can interact with the real world through the use of tools, allowing it to perform tasks beyond what an LLM can do in isolation.\n\n**How to Use an LLM Agent for Search Retrieval**\n\nTo create an LLM agent for search retrieval, you can follow these steps:\n\n1. **Define the tools**: Identify the tools that the agent can use to interact with the outside world. For example, you can create a tool that allows the agent to access a web search engine (e.g., DuckDuckGo) and a math tool that provides basic calculator functionality.\n2. **Create the LLM agent**: Use a framework like ReAct to create an agent that can reason about its thoughts, take actions, and observe the results. This involves defining the agent\'s name, description, and function (e.g., searc

In [52]:
#adding conversation summary to the llm now
##but do we need memory??##

what we need is. a better retrieval logic that very accurately takes on to the right page(end goal) 
but right now we need that **at least the top 2** has the right one in the search

In [56]:
#current retrieval not great but fast
results = database.similarity_search("agentic RAG", k=2)
for doc in results:
    page = doc.metadata.get("page")
    print(f"Page: {page}\nContent: {doc.page_content}\n")

Page: 318
Content: Figur e 8-3. A RAG system formulates an answer to a question and (pr eferably) cites its information
sources.
The rest of the chapter covers these three types of systems in more detail.
While these are the major categories, they are not the only LLM
applications in the domain of search.
Semantic Search with Language Models
Let’s now dive into more detail on the major categories of systems that can
upgrade the search capabilities of our language models. We’ll start with
dense retrieval and then move on through reranking and RAG.
Dense Retrieval
Recall that embeddings turn text into numeric representations. Those can be
thought of as points in space, as we can see in Figure 8-4 . Points that are
close together mean that the text they represent is similar . So in this
example, text 1 and text 2 are more similar to each other (because they are
near each other) than text 3 (because it’ s farther away).

Page: 356
Content: Query routing
An additional enhancement is to give

In [ ]:
query={'query':"what is agentic rag"}
actual_query=chain1.invoke(query)


In [72]:
print(actual_query)


Here's a rewritten version of the user query that can help retrieve relevant documents:

**Search Query:** "definition of agentic role" OR "characteristics of agentic behavior"

This rewritten query uses specific keywords and phrases to capture the essence of the original question. By using terms like "definition", "role", and "behavior", we can search for documents that provide a clear explanation of what an "agentic rag" is, or related concepts that describe its characteristics.


In [97]:
#current retrieval not great but fast
results = database.similarity_search("actual_query", k=2)
for doc in results:
    page = doc.metadata.get("page")
    print(f"Page: {page}\nContent: {doc.page_content}\n")

Page: 320
Content: Figur e 8-5. Dense r etrieval r elies on the pr operty that sear ch queries will be close to their r elevant
results.
Judging by the distances in Figure 8-5 , “text 2” is the best result for this
query , followed by “text 1.” Two questions could arise here, however:
Should text 3 even be returned as a result? That’ s a decision for
you, the system designer . It’s sometimes desirable to have a max
threshold of similarity score to filter out irrelevant results (in case
the corpus has no relevant results for the query).
Are a query and its best result semantically similar? Not always.
This is why language models need to be trained on question-
answer pairs to become better at retrieval. This process is
explained in more detail in Chapter 10 .
Figure 8-6  shows how we chunk a document before proceeding to embed
each chunk. Those embedding vectors are then stored in the vector database
and are ready for retrieval.

Page: 212
Content:     verbose=True
).fit(abstracts , emb

In [80]:
#implementing keyword search
# from sklearn.feature_extraction import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
import numpy as np
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
from tqdm import tqdm
def tokenizer(text):
    tokenizer_corpus=[]
    for text in text.lower().split():
        text=text.strip(string.punctuation)
    
    if len(text)>0 and text not in _stop_words.ENGLISH_STOP_WORDS:
        tokenizer_corpus.append(text)
    return tokenizer_corpus



In [ ]:
tokenized_doc=[]
for passage in tqdm(page_texts):
    tokenized_doc.append(tokenizer(passage))

bm25= BM25Okapi(tokenized_doc) #notice how naming it tokenize_doc=... will give error later 
def keyword_search(query, top_k=3, num_candidates=10):
    print("input question", query)

    bm_25_scores= bm25.get_scores(tokenizer(query))
    top_n= np.argpartition(bm_25_scores, -num_candidates)[-num_candidates:]
    bm25_hits= [{'corpus_id': idx , 'score':bm_25_scores[idx]} for idx in top_n]
    bm25_hits=sorted(bm25_hits, key= lambda x: x['score'], reverse=True)

    print("top 3 results")
    for hit in bm25_hits[0:top_k]:
        print("\t{:.3f}\t".format(hit['score']),
        page_texts[hit['corpus_id']].replace("\n"," ")) # fetch the content then replace line breaks with space


100%|██████████| 598/598 [00:00<00:00, 3578.73it/s]


In [ ]:


keyword_search("what is agentic RAG")



input question what is agentic RAG
top 3 results
	5.009	 training configuration , Training Configuration quantization , Model I/O: Loading Quantized Models with LangChain , Compressing the model for (more) ef ficient training -Compressing the model for (more) ef ficient training quantization_config parameter , LoRA  Configuration quantized low-rank adaptation  (see QLoRA)  Querying Transformer (Q-Former) , BLIP-2: Bridging the Modality Gap - BLIP-2: Bridging the Modality Gap R r parameter , LoRA  Configuration RAG (retrieval-augmented generation) , Token Embeddings , Text Embeddings (for Sentences and Whole Documents) , Overview of Semantic Search and RAG , Retrieval-Augmented Generation (RAG) -RAG Evaluation agentic RAG , Agentic RAG basic pipeline , Retrieval-Augmented Generation (RAG) converting search system to , From Search to RAG evaluating results , RAG Evaluation grounded generation , Example: Grounded Generation with an LLM API with local models , Example: RAG with Local Model

**langchain stores the text in doc.page_content and metadata in .metadata**

In [ ]:

results = [
    {
    'page-number': doc.metadata.get("page"),
    'score' : score,
    'text': doc.page_content
    }

    for doc, score in database.similarity_search_with_relevance_scores("what is agentic rag", k=3)
]
print(results)


[{'page-number': 318, 'score': 0.8240956707105538, 'text': 'Figur e 8-3. A RAG system formulates an answer to a question and (pr eferably) cites its information\nsources.\nThe rest of the chapter covers these three types of systems in more detail.\nWhile these are the major categories, they are not the only LLM\napplications in the domain of search.\nSemantic Search with Language Models\nLet’s now dive into more detail on the major categories of systems that can\nupgrade the search capabilities of our language models. We’ll start with\ndense retrieval and then move on through reranking and RAG.\nDense Retrieval\nRecall that embeddings turn text into numeric representations. Those can be\nthought of as points in space, as we can see in Figure 8-4 . Points that are\nclose together mean that the text they represent is similar . So in this\nexample, text 1 and text 2 are more similar to each other (because they are\nnear each other) than text 3 (because it’ s farther away).'}, {'page-numbe

In [92]:
# from langchain.retrievers.weaviate_hybrid_search import WeaviateHybridSearchRetriever
# retriever = WeaviateHybridSearchRetriever(
#     alpha = 0.5,               # defaults to 0.5, which is equal weighting between keyword and semantic search
#     client = query,           # keyword arguments to pass to the Weaviate client
#     index_name = "page_ids",  # The name of the index to use
#     text_key = "page_texts",         # The name of the text key to use
#     attributes = [],           # The attributes to return in the results
# )